# CSC_52081_EP - Lab 09 - ADVERSARIAL BANDITS

NOTE: The beginning of this lab is exactly the same as lab 3. The task for lab 9 is only the EXP implementation at the end of the lab. Still, it uses things that you have implemented in lab 3 for evaluation / comparison so you should first copy-paste your solutions of lab 3. 

### Main Objectives 

Today we will study bandits in the context of sequential decision making, and the exploration-exploitation tradeoff. You may find the required background in the lecture slides, lecture notes, and the references provided within. 
 
### Instructions

Work your way through the notebook, and provide code where indicated by `# TODO` to complete the tasks. Check Moodle for details on how to submit your work.

### Imports

First, we're going to import `numpy` and some utility functions/classes that we will use. Make sure the `bandit_machines.py` is in your working directory. 

In [ ]:
import numpy as np
import scipy

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from bandit_machines import BernoulliMAB, GaussianMAB

In [ ]:
def randmax(a):
    """ return a random maximum """
    a = np.array(a)  
    max_indices = np.flatnonzero(a == a.max())
    return np.random.choice(max_indices)

### Creating a bandit environment. 

What do we mean by a bandit environment or bandit machine? It's a kind of state-less enviornment. When generates from a particular generation, depending on which arm (or, action) is pulled (or, taken). 

In [ ]:
n_arms = 2
env_B = BernoulliMAB(n_arms=2,labels=["Arm 1","Arm 2"],means=[0.5,0.7])
# 

### Evaluating a bandit learner

The function given below executes one bandit algorithm on one multi-armed-bandit (MAB) instance. We will use this to compare bandit algorithms. Note that we compare them on a basis of **cumulative regret**. In this lab we are mainly interested in an empirical analysis of bandits. 

In [ ]:
def evaluate_run(env, pi, T=1000):
    '''
    Run a bandit agent on a bandit instance (environment) for T steps.
    '''
    r_log = []
    a_log = []
    pi.clear()
    for t in range(T):
        a = pi.act()
        r = env.rwd(a)
        pi.update(a,r)
        r_log.append(r)
        a_log.append(a)
    return a_log, r_log

In [ ]:
def cumulative_regret(env, selections):
    """Compute the pseudo-regret associated with a sequence of arm selections"""
    best = np.max(env.means)*np.ones(len(selections))
    real = np.array(env.means)[selections]
    return np.cumsum(best - real)

### Baselines

In the following, we implemented two naive bandit strategies: the greedy strategy (or Follow-the-Leader, `FTL`) and a strategy that explores arms uniformly at random (`UniformExploration`). 

Take note of the `class` structure (methods implemented), you will need to use the same structure in your implementations shortly. 

In [ ]:
class FTL:

    """Follow the Leader (a.k.a. greedy strategy)"""

    def __init__(self,n_arms):
        self.n_arms = n_arms
        self.clear()

    def clear(self):
        self.nbDraws = np.zeros(self.n_arms)
        self.cumRewards = np.zeros(self.n_arms)
    
    def act(self):
        if (min(self.nbDraws)==0):
            return randmax(-self.nbDraws)
        else:
            return randmax(self.cumRewards/self.nbDraws)

    def update(self,a,r):
        self.cumRewards[a] = self.cumRewards[a] + r
        self.nbDraws[a] = self.nbDraws[a] + 1

    def name(self):
        return "FTL"

In [ ]:
class UniformExploration:

    """a strategy that uniformly explores arms"""

    def __init__(self,n_arms):
        self.n_arms = n_arms
        self.clear()

    def clear(self):
        self.nbDraws = np.zeros(self.n_arms)
        self.cumRewards = np.zeros(self.n_arms)
    
    def act(self):
        return np.random.randint(0,self.n_arms)

    def update(self,arm,reward):
        self.cumRewards[arm] = self.cumRewards[arm]+reward
        self.nbDraws[arm] = self.nbDraws[arm] +1

    def name(self):
        return "Uniform"

Below we run `FTL` on the simple Bernoulli bandit instance defined above, and we visualize its behavior.

In [ ]:
ftl = FTL(n_arms)
T = 200
actions, rewards = evaluate_run(env_B, ftl, T)
regret1 = cumulative_regret(env_B, actions)
# Histogram of the number of arms selections
plt.clf()
plt.xlabel("Arms", fontsize=14)
plt.xticks(range(ftl.n_arms))
plt.ylabel("Number of arms selections", fontsize=14)
plt.hist(actions, np.max(actions) + 1)
plt.title("Number of selections of each arm", fontsize=14)
plt.show()

In [ ]:
# Cumulative regret as a function of time
plt.figure(2)
plt.clf()
plt.xlabel("$t$", fontsize=14)
plt.ylabel("Cumulative regret", fontsize=14)
plt.title("Regret as a function of time")
plt.plot(range(T), regret1, 'black', linewidth=1)
plt.show()

Let's compare regret between FTL and the Uniform algorithm.

You can run the code more than once, and you should get a different result each time.

In [ ]:
# Compare their regret
ufm = UniformExploration(n_arms)
T = 200
actions, rewards = evaluate_run(env_B, ufm, T)
regret2 = cumulative_regret(env_B, actions)

In [ ]:
plt.xlabel("Time steps", fontsize=14)
plt.ylabel("Cumulative regret", fontsize=14)
plt.title("Regret as a function of time")
plt.plot(range(0, T), regret1,label=ftl.name())
plt.plot(range(0, T), regret2,label=ufm.name())
plt.legend()
plt.show()

### Comparison averaged over multiple runs

Since the regret is defined as an **expectation**, we need several runs to estimate its value. We can also take a look at the distribution of the pseudo-regret. The function below gathers results accross multiple runs.

In [ ]:
def evaluate_runs(env, pi, T=1000, N=10):
    '''
        Parameters
        ----------

        env : 
            bandit machine
        pi : 
            bandit algorithm
        N : int
            number of experiments
        T : int
            number of trails per experiment
    '''
    R_log = np.zeros((N,T))
    A_log = np.zeros((N,T),dtype=int)
    for n in range(N):
        np.random.seed()
        A_log[n], R_log[n] = evaluate_run(env, pi, T)

    return A_log, R_log

The following function will be useful for comparing arbitrary combinations of bandit algorithms empirically under a number (`N`) of runs. 

In [ ]:
def plot_results(env, algorithms, T=1000, N=10, fname=None): 

    plt.clf()
    plt.title("Cumulative Regret over %d Runs" % N)

    colours = ['b', 'r', 'g', 'm']

    for j, pi in enumerate(algorithms):

        A_log, R_log = evaluate_runs(env, pi, T, N)
        Regret = np.array([cumulative_regret(env,A_log[n,:]) for n in range(N)])
        meanRegret = np.mean(Regret, 0)
        upperQuantile = np.quantile(Regret, 0.95, 0) 
        lowerQuantile = np.quantile(Regret, 0.05, 0)

        plt.plot(range(T), meanRegret, linewidth=3.0, color=colours[j], label=""+pi.name())
        plt.plot(range(T), upperQuantile, linestyle="dashed", color=colours[j])
        plt.plot(range(T), lowerQuantile, linestyle="dashed", color=colours[j])

    plt.xlabel("$t$", fontsize=10)
    plt.ylabel("Cumulative regret", fontsize=10)
    plt.legend()
    plt.tight_layout()
    if fname is not None:
        plt.savefig(fname,bbox_inches='tight')
    plt.show()

So now let's compare FTL and Uniform in terms of expectation. What do you observe? 

In [ ]:
plot_results(env_B, [ftl, ufm])

### Task 

Implement Epsilon Greedy ($\epsilon$) in the `EpsilonGreedy` class below, with the same structure (method definitions) as `FTL` and `Uniform` above.

In [ ]:
class EpsilonGreedy:

    """ Epsilon Greedy """

    def __init__(self, n_arms, epsilon=0.05):
        self.n_arms = n_arms
        self.epsilon = epsilon
        self.clear()
        # TODO  
    def name(self):
        return "e-Greedy(%3.2f)" % self.epsilon
    def clear(self):
        self.nbDraws = np.zeros(self.n_arms)
        self.cumRewards = np.zeros(self.n_arms)
    
    def act(self):
        if (np.random.rand() < self.epsilon):
            return np.random.randint(0,self.n_arms)
        else:
            if (min(self.nbDraws)==0):
                return randmax(-self.nbDraws)
            else:
                return randmax(self.cumRewards/self.nbDraws)
    
    def update(self,arm,reward):
        self.cumRewards[arm] = self.cumRewards[arm]+reward
        self.nbDraws[arm] = self.nbDraws[arm] +1
        

Have a look at the performance of this method for different values of $\epsilon$. 

In [ ]:
plot_results(env_B, [EpsilonGreedy(env_B.n_arms,0),EpsilonGreedy(env_B.n_arms,0.1),EpsilonGreedy(env_B.n_arms,0.01)], T=100, N=100)
# 

# And with another bandit instance (Gaussian arms, this time)
env_G = GaussianMAB(2)
plot_results(env_G, [EpsilonGreedy(env_G.n_arms,0),EpsilonGreedy(env_G.n_arms,0.1),EpsilonGreedy(env_G.n_arms,0.01)], T=100, N=100)

### Task 

Implement UCB($\alpha$) in the `UCB` class below, with the same structure (method definitions) as `FTL` and `Uniform` above.

In [ ]:
class UCB():


    def __init__(self,n_arms,alpha=1/2):
        self.n_arms = n_arms
        self.alpha = alpha
        self.clear()
        # TODO 

    def name(self):
        return "UCB(%3.2f)" % self.alpha
    def clear(self):
        self.nbDraws = np.zeros(self.n_arms)
        self.cumRewards = np.zeros(self.n_arms)
        self.ucbs = np.full(self.n_arms,np.inf)
        self.time_step = 0
    
    def act(self):
        self.time_step = self.time_step + 1
        return randmax(self.ucbs)
    
    def update(self,arm,reward):
        self.cumRewards[arm] = self.cumRewards[arm]+reward
        self.nbDraws[arm] = self.nbDraws[arm] +1
        for a in range(self.n_arms):
            if self.nbDraws[a] > 0:  
                self.ucbs[a] = self.cumRewards[a] / self.nbDraws[a] + np.sqrt(self.alpha * np.log(self.time_step) / self.nbDraws[a])

    def render(self, env_G = None, fname=None):
        arms = np.arange(self.n_arms)
        avg_rewards = np.divide(self.cumRewards, self.nbDraws, where=self.nbDraws > 0, out=np.zeros_like(self.cumRewards))
        confidence_bounds = self.ucbs - avg_rewards  

        plt.figure(figsize=(10, 5))
        plt.bar(arms, avg_rewards, yerr=confidence_bounds, capsize=5, label="Estimated Value ± UCB")
        plt.xlabel("Arm")
        plt.ylabel("Estimated Reward")
        plt.title(f"UCB Estimates (t={self.time_step})")
        plt.legend()
        plt.grid(True)

        if fname:
            plt.savefig(fname)
        else:
            plt.show()


We'll just check that your UCB implementation is working.

In [ ]:
ucb = UCB(env_G.n_arms)
plot_results(env_G, [ucb], T=100, N=10)
ucb.render(env_G) # ,fname="bandit_ucb_trained.pdf"

Let's try a different bandit machine. Namely, we'll try a mixed multi-arm bandit, with arms of different reward distributions, but you could experiment with others also. 

In [ ]:
from bandit_machines import MixedMAB, TruncatedExponential, Bernoulli
# You should play around with different types of bandit environments here 
env_M = MixedMAB([TruncatedExponential(2, 1), Bernoulli(0.3), TruncatedExponential(3.5, 1)])
plot_results(env_M, [ucb], T=100, N=10)

### Task 

Implement Thompson Sampling in the class provided below, with the same structure as the methods you implemented above.

In [ ]:
from scipy.stats import beta

In [ ]:
class ThompsonSampling():

    def __init__(self,n_arms):
        ''' Bernoulli bandit machine (n_arms arms, each giving 0 or 1 with some probability) '''
        self.n_arms = n_arms
        # TODO  
    def name(self):
        return "Thompson"
    
    def clear(self):
        self.alpha = np.ones(self.n_arms)  
        self.beta = np.ones(self.n_arms)  

    def act(self):
        samples = []
        for a in range(self.n_arms):
            samples.append(beta.rvs(self.alpha[a], self.beta[a]))
        return randmax(samples) 

    def update(self, a, r):
        self.alpha[a] += r  
        self.beta[a] += (1 - r)  

Finally, let's compare the three algorithms that you have implemented.

In [ ]:
# 
#
# Here, we carry out the comparison
plot_results(env_B, [UCB(env_B.n_arms), EpsilonGreedy(env_B.n_arms), ThompsonSampling(env_B.n_arms)],N=100)

### Something to Think About 

Regarding the Environment we used for Lab1 and Lab2, we could also use bandits in that scenario, namely, to if we had to decide on the best action to take *without* any observation. When considered as a 'rollout', this takes us closer and closer to Monte Carlo Tree Search and other algorithms in the domain of reinforcement learning. 

# LAB 9 : EXP3

We finally explore the EXP3 algorithm. The EXP3 algorithm was proposed for adversarial bandits, but we will test it here using the stochastic bandits that we have seen above. 

EXP3 was originally proposed by [Auer et al., 1995](https://www.computer.org/csdl/proceedings-article/focs/1995/71830322/12OmNAYGlx6) and subsequently much used, refined and analyzed. An accessible presentation can be found in Chapter 11 of [Lattimore and Szepesvari's book](https://tor-lattimore.com/downloads/book/book.pdf). 

The idea of EXP3 is to use exponential weights based on an importance sampling estimate of the reward of each arm. Specifically, the algorithm does the following steps: 
- Initialize $\hat{S}_{0,i} = 0$ for all arms $i$
- For t = 1...n, 
    - calculate the distribution $P$ as $$P_{ti} = \frac{\exp(\eta \hat{S}_{t-1,i})}{\sum_i \exp(\eta \hat{S}_{t-1,i})}$$
    - sample the arm $A_t$ at time $t$ according to this distribution
    - upon receiving reward $X_t$ after drawing arm $A_t$, update the estimates of every arm using the loss-based importance-weighted estimator: $$\hat{S}_{t,i} = \hat{S}_{t-1,i} + 1 - \frac{1_{A_t = i}\cdot(1 - X_t)}{P_{ti}}$$
    
    
Here $\eta$ is a parameter called the learning rate. We assume that it is constant (i.e., does not depend on $t$). Note that we also assumed that the reward is in $[0, 1]$.

### Task: Implement EXP3 with $\eta$ as a parameter in the class below. 

Make sure to use the `eta` parameter as supplied to the function `__init__`. 

In [ ]:
class EXP3():
    """EXP3 algorithm with parameter eta"""

    def __init__(self, n_arms, eta=1):
        self.n_arms = n_arms
        # TODO  
        self.eta = eta
        self.clear()

    def name(self):
        return "Thompson"
    

    def clear(self):
        self.estimator = np.zeros(self.n_arms)
        self.time_step = 0

    def act(self):
        self.time_step += 1
        epsilon = 1e-10
        self.distribution = np.exp(self.eta * self.estimator) / np.sum(np.exp(self.eta*self.estimator)) + epsilon
        self.distribution /= np.sum(self.distribution)
        return np.random.choice(self.n_arms,p=self.distribution)
    
    def update(self,arm,reward):
        for i in range(self.n_arms):
            self.estimator[i] += 1 - (i==arm)*(1-reward)/self.distribution[i]

### Task: compare the results with the previously studied algorithms from lab 3

In [ ]:
plot_results(env_M, [UCB(env_M.n_arms), EpsilonGreedy(env_M.n_arms), ThompsonSampling(env_M.n_arms), EXP3(env_M.n_arms, eta = 0.5)],N=100)

## Quiz.

There is a pdf sheet associated with the lab with some quiz questions relating to bandits, as covered in the lecture. Replace the empty lists in the `answers` dictionary, such that answers[1] is a list of elements of 'a', 'b', 'c', 'd', 'e', 'f' (and so on, for the other questions).

In [ ]:
answers = {
    1 : ['b','e'],
    2 : ['a','b','c'], 
    3 : ['a','c'], 
    4 : ['b','c','e'], 
    5 : ['a','b','c','d'], 
    6 : ['a','b','d','e'],
    7 : ['b','c','e','f']
}
